In [1]:
import traffic
from traffic.data import opensky
from traffic.core import Traffic
from rich.pretty import pprint
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import h5py
import os

In [2]:
def download_month(month: int, year: int, start_day = 1, cached=True):

    if month < 10:
        str_month = "0" + str(month)
    else:
        str_month = str(month)

    filename = "Frankfurt_LH_" + str(year)[2:] + str_month + ".h5"
    print(filename)
    next_month = month + 1 if month < 12 else  1
    str_next_month = str(next_month) if next_month > 9 else "0" + str(next_month)

    start_day_str = "0" + str(start_day) if start_day < 10 else str(start_day)
    datetime_str_start = str(year) + "-"+ str_month + "-" + start_day_str +  " 00:00:00"
    datetime_str_end = str(year) + "-" + str_next_month + "-01 00:00:00" if next_month != 1 else str(year+1) + "-" + str_next_month + "-01 00:00:00"
    datetime_current = datetime.strptime(datetime_str_start, '%Y-%m-%d %H:%M:%S')
    datetime_end = datetime.strptime(datetime_str_end, '%Y-%m-%d %H:%M:%S')

    while datetime_current < datetime_end:

        print(datetime_current)
        day = datetime_current.day
        if day < 10:
            day_str = "0" + str(day)
        else:
            day_str = str(day)

        h5_key = "LH_22" + str_month + day_str
        print(h5_key)
        str_current_day = str(datetime_current)
        str_next_day = str(datetime_current + timedelta(days = 1))
        print("getting data between "+str_current_day+ " and "+ str_next_day)

        trajectories = opensky.history(str_current_day,
                            stop= str_next_day,
                            arrival_airport ="EDDF",
                            cached=cached)
        try:
            callsigns_lh = trajectories.data.callsign.loc[trajectories.data.callsign.str.contains("DLH").astype(bool)]

            callsigns_lh = callsigns_lh.unique()
            callsigns_lh = callsigns_lh[~pd.isnull(callsigns_lh)]
            trajectories_lh = trajectories[callsigns_lh]
            print("add",h5_key,"to file")

            trajectories_lh.to_hdf(filename, key=h5_key, format = 'table')

        except AttributeError as e:
            print(e)

        datetime_current += timedelta(days = 1)


In [3]:
download_month(2, 2023, 1, cached = False)

Output()

Frankfurt_LH_2302.h5
2023-02-01 00:00:00
LH_220201
getting data between 2023-02-01 00:00:00 and 2023-02-02 00:00:00


add LH_220201 to file


Output()

2023-02-02 00:00:00
LH_220202
getting data between 2023-02-02 00:00:00 and 2023-02-03 00:00:00


...

add LH_220210 to file


Output()

2023-02-11 00:00:00
LH_220211
getting data between 2023-02-11 00:00:00 and 2023-02-12 00:00:00


add LH_220211 to file


HDF5ExtError: HDF5 error back trace

  File "D:\bld\hdf5_split_1671624031008\work\src\H5Dio.c", line 291, in H5Dwrite
    can't write data
  File "D:\bld\hdf5_split_1671624031008\work\src\H5VLcallback.c", line 2113, in H5VL_dataset_write
    dataset write failed
  File "D:\bld\hdf5_split_1671624031008\work\src\H5VLcallback.c", line 2080, in H5VL__dataset_write
    dataset write failed
  File "D:\bld\hdf5_split_1671624031008\work\src\H5VLnative_dataset.c", line 207, in H5VL__native_dataset_write
    can't write data
  File "D:\bld\hdf5_split_1671624031008\work\src\H5Dio.c", line 775, in H5D__write
    can't write data
  File "D:\bld\hdf5_split_1671624031008\work\src\H5Dchunk.c", line 2703, in H5D__chunk_write
    unable to read raw data chunk
  File "D:\bld\hdf5_split_1671624031008\work\src\H5Dchunk.c", line 4017, in H5D__chunk_lock
    unable to preempt chunk(s) from cache
  File "D:\bld\hdf5_split_1671624031008\work\src\H5Dchunk.c", line 3683, in H5D__chunk_cache_prune
    unable to p

In [3]:
with h5py.File("Frankfurt_LH_2204.h5", 'r') as f:
    print(list(f.keys()))

['LH_220401', 'LH_220402', 'LH_220403', 'LH_220404', 'LH_220405', 'LH_220406', 'LH_220407', 'LH_220408', 'LH_220409', 'LH_220410', 'LH_220411', 'LH_220412', 'LH_220413', 'LH_220414', 'LH_220415', 'LH_220416', 'LH_220417', 'LH_220418', 'LH_220419', 'LH_220420', 'LH_220421', 'LH_220422']
